# Backtest vs Live Trading Comparison

## Overview
This notebook validates that live trading results match backtest predictions by comparing:
1. **Bar-level data**: Binarization prices and close prices across the full date range
2. **Order execution**: BUY/SELL order timing and prices on the trading day
3. **Tick-level data**: Granular price movements during each position

## Purpose
Ensure that the backtest accurately represents live market conditions and that any divergence in outcomes is due to strategy logic, not data differences or execution issues.

## Analysis Flow
1. Load backtest data from folder structure (/data/quiescence/backtest)
2. Load live trading data from folder structure (/data/quiescence/live_runs)
3. Compare price series and identify any divergence
4. Validate order execution timing and prices
5. Examine tick-by-tick price movements for each position

In [ ]:
import sys
from pathlib import Path

# Add the analysis directory to the path to import utilities
# The utilities.py is in the analysis folder, and we're in analysis/live_trading/momentum_strategy
analysis_dir = Path.cwd().parent.parent if (Path.cwd() / '../../utilities.py').exists() else Path.cwd()
if str(analysis_dir) not in sys.path:
    sys.path.insert(0, str(analysis_dir))

import os
import json
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from plotly.subplots import make_subplots

from utilities import convert_utc_to_ny

## Configuration

Set the parameters for this analysis:
- **Stock symbol**: The ticker symbol to analyze (e.g., MSFT)
- **Trading day**: The specific date to examine in detail (format: YYYY-MM-DD)
- **Paths**: Backtest and live data root directories

**Note**: The notebook will scan for runs matching the stock symbol and date, then display them for you to select.

In [ ]:
# Just need to change these two variables to load different runs
ticker = "LLY"
trading_day = "2026-01-28"  

In [ ]:
# Project paths - automatically determined from notebook location
STORAGE_ROOT = Path("/data/quiescence_bak/")
BACKTEST_ROOT = STORAGE_ROOT / "backtest"
LIVE_RUNS_ROOT = STORAGE_ROOT / "live_runs"

print(f"Analyzing stock: {ticker}")

print(f"Live runs root: {LIVE_RUNS_ROOT}")
print(f"Backtest root: {BACKTEST_ROOT}")

In [ ]:
# Construct paths to backtest and live run directories
live_run = LIVE_RUNS_ROOT / ticker / trading_day 
backtest_run = live_run / "backtest"

if not backtest_run:
    raise ValueError(f"No backtest runs found at {backtest_date_dir}")
if not live_run:
    raise ValueError(f"No live runs found at {live_date_dir}")

print(f"Backtest directory: {backtest_run}")
print(f"Live directory: {live_run}")

In [ ]:
# Import Nautilus catalog tools
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.data import QuoteTick, TradeTick

# Initialize catalog
CATALOG_PATH = STORAGE_ROOT / "catalog"
catalog = ParquetDataCatalog(str(CATALOG_PATH))
instrument_id = InstrumentId.from_str(f"{ticker}.POLYGON")

print(f"Catalog path: {CATALOG_PATH}")
print(f"Instrument: {instrument_id}")

## Verify Run Parameters Match

Before comparing results, verify that both the backtest and live runs used identical parameters.

In [ ]:
# Load run_parameters.json from both directories
backtest_params_file = backtest_run / "run_parameters.json"
live_params_file = live_run / "run_parameters.json"

with open(backtest_params_file, 'r') as f:
    backtest_params = json.load(f)

with open(live_params_file, 'r') as f:
    live_params = json.load(f)

# Parameters to ignore when comparing (expected to be different between backtest and live)
ignore_keys = {
    'backtest_end_date',
    'date',
    'ibg_host',
    'ibg_port',
    'run_name',
    'run_type',
    'strategy_data_output_path',
    'time',
    'maker_fee',
    'taker_fee',
    'source',
    'burnin_limit'
}

# Find differences excluding ignored keys
all_keys = set(backtest_params.keys()) | set(live_params.keys())
relevant_keys = all_keys - ignore_keys
differences = []

for key in relevant_keys:
    backtest_val = backtest_params.get(key, "MISSING")
    live_val = live_params.get(key, "MISSING")
    
    if backtest_val != live_val:
        differences.append((key, backtest_val, live_val))

# Display results
if len(differences) == 0:
    print("✓ Run parameters match (excluding expected differences)")
    print(f"  Compared {len(relevant_keys)} parameters")
    print(f"  Ignored {len(ignore_keys)} parameters: {', '.join(sorted(ignore_keys))}")
else:
    print(f"✗ Warning: {len(differences)} parameter difference(s) found!")
    print(f"  (Ignoring: {', '.join(sorted(ignore_keys))})")
    print("\nDifferences found:")
    
    for key, backtest_val, live_val in sorted(differences):
        print(f"\n  {key}:")
        print(f"    Backtest: {backtest_val}")
        print(f"    Live:     {live_val}")

## Load Strategy Data (Binarization Prices)

Load the strategy_data.jsonl file from both runs to get bar-by-bar binarization prices.

In [ ]:
# Load backtest strategy data
backtest_strategy_file = backtest_run / "strategy_data.jsonl"
backtest_data = []
with open(backtest_strategy_file, 'r') as f:
    for line in f:
        backtest_data.append(json.loads(line))

df_backtest = pd.DataFrame(backtest_data)
df_backtest['bar_close_local_datetime'] = pd.to_datetime(df_backtest['bar_close_local_datetime'])

# Load live strategy data
live_strategy_file = live_run / "strategy_data.jsonl"
live_data = []
with open(live_strategy_file, 'r') as f:
    for line in f:
        live_data.append(json.loads(line))

df_live = pd.DataFrame(live_data)
df_live['bar_close_local_datetime'] = pd.to_datetime(df_live['bar_close_local_datetime'])

print(f"Backtest: {len(df_backtest)} bars")
print(f"Live: {len(df_live)} bars")

In [ ]:
cols = ['bar_counter', 'bar_close_local_datetime',
       'instrument_id', 'close_price', 'bar_return',
       'binarization_price', 'p_value_long', 'p_value_short', 'signal_value',
       'rb_signal_full', 'burnin_complete', 'current_position_test',
       'pnl_curve_compounded', 'pnl_curve_cash_one_share']

In [ ]:
df_live[cols]

In [ ]:
df_backtest[cols]

## Load Fill Reports

Load actual fills from both backtest and live to compare execution.

In [ ]:
# Load backtest fills (from orders.csv since backtest doesn't have separate fills)
backtest_orders_file = backtest_run / "orders.csv"
df_backtest_orders = pd.read_csv(backtest_orders_file)

if not df_backtest_orders.empty:
    df_backtest_orders['order_time_local'] = df_backtest_orders['ts_init'].apply(
        lambda ts: convert_utc_to_ny(ts / 1e9).replace(tzinfo=None)
    )

print(f"Backtest orders: {len(df_backtest_orders)}")

In [ ]:
# Load live fills
live_fills_file = live_run / "fills.csv"
df_live_fills = pd.read_csv(live_fills_file)

if not df_live_fills.empty:
    df_live_fills['fill_time_local'] = df_live_fills['ts_init'].apply(
        lambda ts: convert_utc_to_ny(ts / 1e9).replace(tzinfo=None)
    )

# Also load live orders for reference
live_orders_file = live_run / "orders.csv"
df_live_orders = pd.read_csv(live_orders_file)

if not df_live_orders.empty:
    df_live_orders['order_time_local'] = df_live_orders['ts_init'].apply(
        lambda ts: convert_utc_to_ny(ts / 1e9).replace(tzinfo=None)
    )

print(f"Live fills: {len(df_live_fills)}")
print(f"Live orders: {len(df_live_orders)}")

## Compare Binarization Price Series

**Validation Goal**: Ensure backtest and live used the same price data for signal generation.

### What is Binarization Price?
The price level used by the strategy to generate trading signals. This is the fundamental input to the strategy logic.

### What to Look For:
- **Perfect alignment**: Lines should overlap exactly
- **Divergence**: Any gap indicates different data sources or processing
- **Time alignment**: Both series should have matching timestamps

If binarization prices diverge, the backtest is invalid because it used different input data than live trading.

In [ ]:
# Convert bar_close_local_datetime to datetime if needed
df_backtest['bar_close_local_datetime'] = pd.to_datetime(df_backtest['bar_close_local_datetime'])
df_live['bar_close_local_datetime'] = pd.to_datetime(df_live['bar_close_local_datetime'])

# Create the comparison plot
fig = go.Figure()

# Add backtest binarization price
fig.add_trace(go.Scatter(
    x=df_backtest['bar_close_local_datetime'],
    y=df_backtest['binarization_price'],
    mode='lines',
    name='Backtest',
    line=dict(color='blue', width=2),
    hovertemplate='<b>Backtest</b><br>Time: %{x}<br>Binarization Price: %{y:.2f}<extra></extra>'
))

# Add live binarization price
fig.add_trace(go.Scatter(
    x=df_live['bar_close_local_datetime'],
    y=df_live['binarization_price'],
    mode='lines',
    name='Live Trading',
    line=dict(color='red', width=2),
    hovertemplate='<b>Live Trading</b><br>Time: %{x}<br>Binarization Price: %{y:.2f}<extra></extra>'
))

# Update layout
fig.update_layout(
    title=f'{ticker} - Binarization Price: Backtest vs Live Trading',
    xaxis_title='Date/Time',
    yaxis_title='Binarization Price',
    hovermode='x unified',
    height=600,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## Position Summary Table

**Diagnostic Analysis**: Examine all positions to identify multi-day position holding patterns.

This table shows:
- **Entry/Exit Times**: When positions were opened and closed
- **Entry/Exit Dates**: The calendar date for each action
- **Multi-Day Flag**: Whether the position was held overnight
- **Duration**: How long the position was held
- **Prices & P&L**: Entry/exit prices and profit/loss

**Expected Behavior**: For an intraday strategy, ALL positions should have the same entry_date and exit_date (no multi-day flags).

**Bug Indicator**: If we see multi-day positions, it means the strategy is NOT closing positions before market close as intended.

In [ ]:
# Extract positions from orders for detailed analysis
def extract_positions_with_dates(df_orders, df_fills=None, label=""):
    """
    Extract position pairs with detailed date/time information for debugging.
    """
    positions = []
    df_sorted = df_orders.sort_values('order_time_local').copy()
    open_order = None
    
    for idx, row in df_sorted.iterrows():
        if row['side'] == 'BUY' and open_order is None:
            open_order = row
        elif row['side'] == 'SELL' and open_order is not None:
            if df_fills is not None:
                # Use fills data for accurate P&L
                entry_order_id = open_order.get('order_id', open_order.get('client_order_id', None))
                entry_fills = df_fills[df_fills['order_id'] == entry_order_id]
                exit_order_id = row.get('order_id', row.get('client_order_id', None))
                exit_fills = df_fills[df_fills['order_id'] == exit_order_id]
                
                if len(entry_fills) > 0:
                    entry_price = (entry_fills['price'] * entry_fills['quantity']).sum() / entry_fills['quantity'].sum()
                    entry_qty = entry_fills['quantity'].sum()
                else:
                    entry_price = open_order.get('avg_px', open_order.get('price', None))
                    entry_qty = open_order.get('filled_qty', 100)
                
                if len(exit_fills) > 0:
                    exit_price = (exit_fills['price'] * exit_fills['quantity']).sum() / exit_fills['quantity'].sum()
                    exit_qty = exit_fills['quantity'].sum()
                else:
                    exit_price = row.get('avg_px', row.get('price', None))
                    exit_qty = row.get('filled_qty', 100)
                
                quantity = min(entry_qty, exit_qty)
            else:
                # Use order data
                entry_price = open_order.get('avg_px', open_order.get('price', None))
                exit_price = row.get('avg_px', row.get('price', None))
                quantity = open_order.get('filled_qty', 100)
            
            entry_time = open_order['order_time_local']
            exit_time = row['order_time_local']
            entry_date = entry_time.date()
            exit_date = exit_time.date()
            is_multiday = entry_date != exit_date
            duration = exit_time - entry_time
            price_diff = exit_price - entry_price
            trade_pnl = price_diff * quantity
            
            positions.append({
                'entry_time': entry_time,
                'exit_time': exit_time,
                'is_multiday': is_multiday,
                'duration': duration,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'price_diff': price_diff,
                'quantity': quantity,
                'trade_pnl': trade_pnl
            })
            open_order = None
    
    df_positions = pd.DataFrame(positions)
    
    if len(df_positions) > 0:
        df_positions['cumulative_pnl'] = df_positions['trade_pnl'].cumsum()
        
        # Print summary
        multiday_count = df_positions['is_multiday'].sum()
        total_count = len(df_positions)
        
        print(f"\n{'='*70}")
        print(f"{label} POSITION ANALYSIS")
        print(f"{'='*70}")
        print(f"Total positions: {total_count}")
        
        if multiday_count > 0:
            print(f"\n⚠️  WARNING: {multiday_count} positions held overnight!")
            print(f"   This indicates the strategy is NOT closing positions before market close.")
            
            # Show examples of multi-day positions
            multiday_positions = df_positions[df_positions['is_multiday']]
            print(f"\n   Multi-day position examples:")
            for idx, pos in multiday_positions.head(5).iterrows():
                print(f"   - Entry: {pos['entry_time']} → Exit: {pos['exit_time']} (Duration: {pos['duration']})")
        else:
            print(f"✓ All positions closed intraday")
        
        print(f"{'='*70}\n")
    
    return df_positions

# Extract and analyze positions
print("Analyzing BACKTEST positions...")
df_backtest_positions_detailed = extract_positions_with_dates(df_backtest_orders, label="BACKTEST")

print("\nAnalyzing LIVE positions...")
df_live_positions_detailed = extract_positions_with_dates(df_live_orders, df_live_fills, label="LIVE")

# Display the tables with formatted times and prices
print("\n" + "="*70)
print("BACKTEST POSITIONS TABLE")
print("="*70)
if len(df_backtest_positions_detailed) > 0:
    # Format for display (excluding date and multiday columns)
    display_cols = ['entry_time', 'exit_time', 'duration', 'entry_price', 'exit_price', 
                   'price_diff', 'quantity', 'trade_pnl', 'cumulative_pnl']
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_columns', None)
    
    # Create a copy for display with formatted columns
    df_display = df_backtest_positions_detailed[display_cols].copy()
    
    # Format time columns as HH:MM:SS.SS
    df_display['entry_time'] = df_display['entry_time'].dt.strftime('%H:%M:%S.%f').str[:-4]
    df_display['exit_time'] = df_display['exit_time'].dt.strftime('%H:%M:%S.%f').str[:-4]
    
    # Format duration as HH:MM:SS.SS
    df_display['duration'] = df_display['duration'].apply(
        lambda x: f"{int(x.total_seconds() // 3600):02d}:{int((x.total_seconds() % 3600) // 60):02d}:{x.total_seconds() % 60:05.2f}"
    )
    
    # Format prices to 4 decimal places
    df_display['entry_price'] = df_display['entry_price'].apply(lambda x: f"{x:.4f}")
    df_display['exit_price'] = df_display['exit_price'].apply(lambda x: f"{x:.4f}")
    df_display['price_diff'] = df_display['price_diff'].apply(lambda x: f"{x:+.4f}")
    
    print(df_display.to_string(index=False))
else:
    print("No backtest positions found")

print("\n" + "="*70)
print("LIVE POSITIONS TABLE")
print("="*70)
if len(df_live_positions_detailed) > 0:
    # Create a copy for display with formatted columns
    df_display = df_live_positions_detailed[display_cols].copy()
    
    # Format time columns as HH:MM:SS.SS
    df_display['entry_time'] = df_display['entry_time'].dt.strftime('%H:%M:%S.%f').str[:-4]
    df_display['exit_time'] = df_display['exit_time'].dt.strftime('%H:%M:%S.%f').str[:-4]
    
    # Format duration as HH:MM:SS.SS
    df_display['duration'] = df_display['duration'].apply(
        lambda x: f"{int(x.total_seconds() // 3600):02d}:{int((x.total_seconds() % 3600) // 60):02d}:{x.total_seconds() % 60:05.2f}"
    )
    
    # Format prices to 4 decimal places
    df_display['entry_price'] = df_display['entry_price'].apply(lambda x: f"{x:.4f}")
    df_display['exit_price'] = df_display['exit_price'].apply(lambda x: f"{x:.4f}")
    df_display['price_diff'] = df_display['price_diff'].apply(lambda x: f"{x:+.4f}")
    
    print(df_display.to_string(index=False))
else:
    print("No live positions found")

In [ ]:
# Create scatter plot of P&L vs Duration
if len(df_backtest_positions_detailed) > 0 or len(df_live_positions_detailed) > 0:
    fig = go.Figure()
    
    # Add backtest positions
    if len(df_backtest_positions_detailed) > 0:
        # Convert duration to minutes for easier interpretation
        backtest_duration_minutes = df_backtest_positions_detailed['duration'].dt.total_seconds() / 60
        
        # Prepare custom data for hover (entry and exit times)
        backtest_hover_data = df_backtest_positions_detailed[['entry_time', 'exit_time']].values
        
        fig.add_trace(go.Scatter(
            x=backtest_duration_minutes,
            y=df_backtest_positions_detailed['trade_pnl'],
            mode='markers',
            name='Backtest',
            marker=dict(
                size=10,
                color='blue',
                opacity=0.6,
                line=dict(width=1, color='darkblue')
            ),
            customdata=backtest_hover_data,
            hovertemplate='<b>Backtest</b><br>Entry: %{customdata[0]}<br>Exit: %{customdata[1]}<br>Duration: %{x:.1f} min<br>P&L: $%{y:.2f}<extra></extra>'
        ))
    
    # Add live positions
    if len(df_live_positions_detailed) > 0:
        # Convert duration to minutes for easier interpretation
        live_duration_minutes = df_live_positions_detailed['duration'].dt.total_seconds() / 60
        
        # Prepare custom data for hover (entry and exit times)
        live_hover_data = df_live_positions_detailed[['entry_time', 'exit_time']].values
        
        fig.add_trace(go.Scatter(
            x=live_duration_minutes,
            y=df_live_positions_detailed['trade_pnl'],
            mode='markers',
            name='Live',
            marker=dict(
                size=10,
                color='red',
                opacity=0.6,
                line=dict(width=1, color='darkred')
            ),
            customdata=live_hover_data,
            hovertemplate='<b>Live</b><br>Entry: %{customdata[0]}<br>Exit: %{customdata[1]}<br>Duration: %{x:.1f} min<br>P&L: $%{y:.2f}<extra></extra>'
        ))
    
    # Add horizontal line at y=0 to show break-even
    fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1)
    
    # Update layout
    fig.update_layout(
        title=f'{ticker} - Trade P&L vs Duration (Correlation Analysis)',
        xaxis_title='Position Duration (minutes)',
        yaxis_title='Trade P&L ($)',
        hovermode='closest',
        height=600,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    fig.show()
    
    # Calculate and display correlation statistics
    print(f"\n{'='*70}")
    print(f"P&L vs DURATION CORRELATION ANALYSIS")
    print(f"{'='*70}")
    
    if len(df_backtest_positions_detailed) > 0:
        backtest_corr = df_backtest_positions_detailed['duration'].dt.total_seconds().corr(
            df_backtest_positions_detailed['trade_pnl']
        )
        print(f"\nBacktest:")
        print(f"  Correlation coefficient: {backtest_corr:.4f}")
        print(f"  Interpretation: {'Negative' if backtest_corr < 0 else 'Positive'} correlation")
        if backtest_corr < -0.3:
            print(f"  ⚠️  Strong negative correlation - longer trades tend to lose money!")
        elif backtest_corr < 0:
            print(f"  Weak negative correlation - slight tendency for longer trades to lose money")
    
    if len(df_live_positions_detailed) > 0:
        live_corr = df_live_positions_detailed['duration'].dt.total_seconds().corr(
            df_live_positions_detailed['trade_pnl']
        )
        print(f"\nLive:")
        print(f"  Correlation coefficient: {live_corr:.4f}")
        print(f"  Interpretation: {'Negative' if live_corr < 0 else 'Positive'} correlation")
        if live_corr < -0.3:
            print(f"  ⚠️  Strong negative correlation - longer trades tend to lose money!")
        elif live_corr < 0:
            print(f"  Weak negative correlation - slight tendency for longer trades to lose money")
    
    print(f"{'='*70}")
else:
    print("No position data available for P&L vs Duration analysis")

## Single Trading Day Analysis with Orders

**Validation Goal**: Verify that orders were executed at the correct times and prices on the specific trading day.

### Visualization Elements:
1. **Price Lines**: 
   - Blue (cornflowerblue): Backtest close prices
   - Red (indianred): Live close prices
2. **Order Markers**:
   - Green triangles (up): BUY orders
   - Orange/Red triangles (down): SELL orders
   - Darker colors: Backtest orders
   - Lighter colors: Live orders

### What to Look For:
- **Order timing**: Do backtest and live orders occur at similar times?
- **Price levels**: Are orders filled at comparable prices?
- **Order count**: Same number of trades in backtest vs live?
- **Price alignment**: Do the close price lines overlap?

Significant differences in order timing or prices indicate execution differences between backtest and live.

In [ ]:
# Check what dates are available in each dataset
print("Backtest date range:")
print(f"  Start: {df_backtest['bar_close_local_datetime'].min()}")
print(f"  End: {df_backtest['bar_close_local_datetime'].max()}")
print(f"  Unique dates: {df_backtest['bar_close_local_datetime'].dt.date.nunique()}")

print("\nLive date range:")
print(f"  Start: {df_live['bar_close_local_datetime'].min()}")
print(f"  End: {df_live['bar_close_local_datetime'].max()}")
print(f"  Unique dates: {df_live['bar_close_local_datetime'].dt.date.nunique()}")

# Filter data to the specific trading day
df_backtest_day = df_backtest[df_backtest['bar_close_local_datetime'].dt.date == pd.to_datetime(trading_day).date()].copy()
df_live_day = df_live[df_live['bar_close_local_datetime'].dt.date == pd.to_datetime(trading_day).date()].copy()

print(f"\nFiltered to {trading_day}:")
print(f"  Backtest bars: {len(df_backtest_day)}")
print(f"  Live bars: {len(df_live_day)}")

if len(df_backtest_day) == 0:
    print(f"\n⚠️  WARNING: No backtest data for {trading_day}!")
    print("   Available backtest dates:")
    for date in sorted(df_backtest['bar_close_local_datetime'].dt.date.unique()):
        print(f"     - {date}")

if len(df_live_day) == 0:
    print(f"\n⚠️  WARNING: No live data for {trading_day}!")
    print("   Available live dates:")
    for date in sorted(df_live['bar_close_local_datetime'].dt.date.unique()):
        print(f"     - {date}")

if len(df_backtest_day) == 0 or len(df_live_day) == 0:
    print("\n❌ Cannot create comparison chart - missing data for selected date")
else:
    # Create the comparison plot for single trading day
    fig = go.Figure()

    # Add backtest close price (medium blue)
    fig.add_trace(go.Scatter(
        x=df_backtest_day['bar_close_local_datetime'],
        y=df_backtest_day['close_price'],
        mode='lines+markers',
        name='Backtest',
        line=dict(color='cornflowerblue', width=2),
        marker=dict(size=4),
        hovertemplate='<b>Backtest</b><br>Time: %{x}<br>Close Price: $%{y:.2f}<extra></extra>'
    ))

    # Add live close price (medium red, solid line)
    fig.add_trace(go.Scatter(
        x=df_live_day['bar_close_local_datetime'],
        y=df_live_day['close_price'],
        mode='lines+markers',
        name='Live Trading',
        line=dict(color='indianred', width=2),
        marker=dict(size=4),
        hovertemplate='<b>Live Trading</b><br>Time: %{x}<br>Close Price: $%{y:.2f}<extra></extra>'
    ))

    # Filter orders to the trading day
    df_backtest_orders_day = df_backtest_orders[df_backtest_orders['order_time_local'].dt.date == pd.to_datetime(trading_day).date()].copy()
    df_live_orders_day = df_live_orders[df_live_orders['order_time_local'].dt.date == pd.to_datetime(trading_day).date()].copy()

    # Add backtest orders to the chart
    if len(df_backtest_orders_day) > 0:
        
        # Merge with price data to get close_price at order time
        df_backtest_orders_with_price = pd.merge_asof(
            df_backtest_orders_day.sort_values('order_time_local'),
            df_backtest_day[['bar_close_local_datetime', 'close_price']].sort_values('bar_close_local_datetime'),
            left_on='order_time_local',
            right_on='bar_close_local_datetime',
            direction='nearest'
        )
        
        # Separate buy and sell orders
        buys = df_backtest_orders_with_price[df_backtest_orders_with_price['side'] == 'BUY']
        sells = df_backtest_orders_with_price[df_backtest_orders_with_price['side'] == 'SELL']
        
        if len(buys) > 0:
            fig.add_trace(go.Scatter(
                x=buys['order_time_local'],
                y=buys['close_price'],
                mode='markers',
                name='Backtest Buy',
                marker=dict(size=12, color='green', symbol='triangle-up', line=dict(width=2, color='darkgreen')),
                hovertemplate='<b>Backtest BUY</b><br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ))
        
        if len(sells) > 0:
            fig.add_trace(go.Scatter(
                x=sells['order_time_local'],
                y=sells['close_price'],
                mode='markers',
                name='Backtest Sell',
                marker=dict(size=12, color='orange', symbol='triangle-down', line=dict(width=2, color='darkorange')),
                hovertemplate='<b>Backtest SELL</b><br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ))

    # Add live orders to the chart
    if len(df_live_orders_day) > 0:
        
        # Merge with price data to get close_price at order time
        df_live_orders_with_price = pd.merge_asof(
            df_live_orders_day.sort_values('order_time_local'),
            df_live_day[['bar_close_local_datetime', 'close_price']].sort_values('bar_close_local_datetime'),
            left_on='order_time_local',
            right_on='bar_close_local_datetime',
            direction='nearest'
        )
        
        # Separate buy and sell orders
        buys = df_live_orders_with_price[df_live_orders_with_price['side'] == 'BUY']
        sells = df_live_orders_with_price[df_live_orders_with_price['side'] == 'SELL']
        
        if len(buys) > 0:
            fig.add_trace(go.Scatter(
                x=buys['order_time_local'],
                y=buys['close_price'],
                mode='markers',
                name='Live Buy',
                marker=dict(size=12, color='lightgreen', symbol='triangle-up', line=dict(width=2, color='green')),
                hovertemplate='<b>Live BUY</b><br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ))
        
        if len(sells) > 0:
            fig.add_trace(go.Scatter(
                x=sells['order_time_local'],
                y=sells['close_price'],
                mode='markers',
                name='Live Sell',
                marker=dict(size=12, color='lightsalmon', symbol='triangle-down', line=dict(width=2, color='red')),
                hovertemplate='<b>Live SELL</b><br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
            ))

    # Update layout
    fig.update_layout(
        title=f'{ticker} - Close Price on {trading_day}: Backtest vs Live Trading (with Orders)',
        xaxis_title='Time',
        yaxis_title='Close Price ($)',
        hovermode='x unified',
        height=600,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        xaxis=dict(
            tickformat='%H:%M',
            dtick=3600000  # 1 hour in milliseconds
        )
    )

    fig.show()

## Position-by-Position Tick Data Analysis

**Validation Goal**: Examine the granular, tick-by-tick price movements during each position to ensure backtest and live experienced identical market conditions.

### Why Tick Data?
Bar data (1-minute aggregations) can hide important intra-bar volatility. Tick data shows every price update and trade execution, providing the most detailed view of market conditions.

### Analysis Approach:
1. **Extract Positions**: Pair up BUY→SELL orders to identify complete position lifecycles
2. **Load Tick Data**: Retrieve all trade ticks from catalog for each position's timeframe (±1 minute buffer)
3. **Side-by-Side Comparison**: Plot backtest (left) vs live (right) for each position
4. **Visual Markers**:
   - Green dashed line: Position entry (BUY)
   - Red dashed line: Position exit (SELL)

### What to Look For:
- **Price trajectory alignment**: Do the tick patterns match?
- **Entry/exit timing**: Are positions opened/closed at the same times?
- **Volatility patterns**: Similar price movements during the position?
- **Tick density**: Comparable number of ticks in both scenarios?

If tick patterns diverge, it indicates different market data between backtest and live, which would invalidate the backtest predictions.

### Initialize Nautilus Catalog
Set up the Nautilus data catalog to access historical tick data from the Polygon data provider.

In [ ]:
# Extract positions from backtest orders (pair up BUY and SELL)
def extract_positions(df_orders, df_fills=None):
    """
    Extract position pairs (entry and exit) from orders dataframe.
    
    Args:
        df_orders: DataFrame with order data
        df_fills: Optional DataFrame with fill data for accurate P&L calculation.
                 If provided, will use actual fill prices/quantities instead of order data.
    
    Returns:
        DataFrame with position information including P&L and duration
    """
    positions = []
    
    # Sort by time to get chronological order
    df_sorted = df_orders.sort_values('order_time_local').copy()
    
    # Track open position
    open_order = None
    
    for idx, row in df_sorted.iterrows():
        if row['side'] == 'BUY' and open_order is None:
            # Opening long position
            open_order = row
        elif row['side'] == 'SELL' and open_order is not None:
            # Closing long position
            
            if df_fills is not None:
                # Use fills data for accurate P&L calculation
                # Get all fills for the entry (BUY) order
                entry_order_id = open_order.get('order_id', open_order.get('client_order_id', None))
                entry_fills = df_fills[df_fills['order_id'] == entry_order_id]
                
                # Get all fills for the exit (SELL) order
                exit_order_id = row.get('order_id', row.get('client_order_id', None))
                exit_fills = df_fills[df_fills['order_id'] == exit_order_id]
                
                # Calculate weighted average prices and total quantities from fills
                if len(entry_fills) > 0:
                    entry_price = (entry_fills['price'] * entry_fills['quantity']).sum() / entry_fills['quantity'].sum()
                    entry_qty = entry_fills['quantity'].sum()
                else:
                    # Fallback to order data if no fills found
                    entry_price = open_order.get('avg_px', open_order.get('price', None))
                    entry_qty = open_order.get('filled_qty', 100)
                
                if len(exit_fills) > 0:
                    exit_price = (exit_fills['price'] * exit_fills['quantity']).sum() / exit_fills['quantity'].sum()
                    exit_qty = exit_fills['quantity'].sum()
                else:
                    # Fallback to order data if no fills found
                    exit_price = row.get('avg_px', row.get('price', None))
                    exit_qty = row.get('filled_qty', 100)
                
                # Use the minimum quantity (should be the same, but safety check)
                quantity = min(entry_qty, exit_qty)
                
            else:
                # Use order data (for backtest or when fills not available)
                entry_price = open_order.get('avg_px', open_order.get('price', None))
                exit_price = row.get('avg_px', row.get('price', None))
                quantity = open_order.get('filled_qty', 100)
            
            # Calculate trade P&L
            trade_pnl = (exit_price - entry_price) * quantity
            
            # Calculate duration
            entry_time = open_order['order_time_local']
            exit_time = row['order_time_local']
            duration = exit_time - entry_time
            duration_minutes = duration.total_seconds() / 60
            
            positions.append({
                'entry_time': entry_time,
                'exit_time': exit_time,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'quantity': quantity,
                'trade_pnl': trade_pnl,
                'duration': duration,
                'duration_minutes': duration_minutes,
                'side': 'LONG'
            })
            open_order = None
    
    df_positions = pd.DataFrame(positions)
    
    # Add cumulative P&L column
    if len(df_positions) > 0:
        df_positions['cumulative_pnl'] = df_positions['trade_pnl'].cumsum()
    
    return df_positions

# Extract positions for backtest and live
df_backtest_positions = extract_positions(df_backtest_orders)
df_live_positions = extract_positions(df_live_orders, df_live_fills)

print(f"Backtest positions: {len(df_backtest_positions)}")
print(f"Live positions: {len(df_live_positions)}")

# Display the positions
#print("\nBacktest positions:")
#display(df_backtest_positions)
#print("\nLive positions:")
#display(df_live_positions)

## Cumulative P&L Comparison

Compare the cumulative profit/loss over time for backtest vs live trading, along with the underlying stock price movement. This helps validate that both strategies are performing similarly and shows how P&L correlates with price movements.

In [ ]:
# Plot cumulative P&L comparison
from plotly.subplots import make_subplots

if len(df_backtest_positions) > 0 and len(df_live_positions) > 0:
    # Create figure with single plot
    fig = go.Figure()
    
    # Cumulative P&L traces with duration-based color gradient
    fig.add_trace(
        go.Scatter(
            x=df_backtest_positions['exit_time'],
            y=df_backtest_positions['cumulative_pnl'],
            mode='lines+markers',
            name='Backtest P&L',
            line=dict(color='rgba(0, 0, 255, 0.3)', width=2),
            marker=dict(
                size=10,
                color=df_backtest_positions['duration_minutes'],
                colorscale='Blues',
                showscale=True,
                colorbar=dict(
                    title="Duration<br>(minutes)",
                    x=1.15,
                    len=0.5,
                    y=0.75
                ),
                line=dict(width=1, color='darkblue')
            ),
            hovertemplate='<b>Backtest</b><br>Time: %{x}<br>Cumulative P&L: $%{y:.2f}<br>Duration: %{marker.color:.1f} min<extra></extra>'
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_live_positions['exit_time'],
            y=df_live_positions['cumulative_pnl'],
            mode='lines+markers',
            name='Live P&L',
            line=dict(color='rgba(255, 0, 0, 0.3)', width=2),
            marker=dict(
                size=10,
                color=df_live_positions['duration_minutes'],
                colorscale='Reds',
                showscale=True,
                colorbar=dict(
                    title="Duration<br>(minutes)",
                    x=1.15,
                    len=0.5,
                    y=0.25
                ),
                line=dict(width=1, color='darkred')
            ),
            hovertemplate='<b>Live</b><br>Time: %{x}<br>Cumulative P&L: $%{y:.2f}<br>Duration: %{marker.color:.1f} min<extra></extra>'
        )
    )
    
    # Update layout
    fig.update_layout(
        height=600,
        title=f"{ticker} - Cumulative P&L Comparison on {trading_day} (Color = Trade Duration)",
        xaxis_title="Time",
        yaxis_title="Cumulative P&L ($)",
        hovermode='closest',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        xaxis=dict(tickformat='%H:%M')
    )
    
    fig.show()
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print(f"P&L SUMMARY FOR {trading_day}")
    print(f"{'='*60}")
    print(f"\nBacktest:")
    print(f"  Total P&L: ${df_backtest_positions['trade_pnl'].sum():.2f}")
    print(f"  Number of trades: {len(df_backtest_positions)}")
    print(f"  Avg P&L per trade: ${df_backtest_positions['trade_pnl'].mean():.2f}")
    print(f"  Avg duration: {df_backtest_positions['duration_minutes'].mean():.1f} minutes")
    print(f"  Win rate: {(df_backtest_positions['trade_pnl'] > 0).sum() / len(df_backtest_positions) * 100:.1f}%")
    
    print(f"\nLive:")
    print(f"  Total P&L: ${df_live_positions['trade_pnl'].sum():.2f}")
    print(f"  Number of trades: {len(df_live_positions)}")
    print(f"  Avg P&L per trade: ${df_live_positions['trade_pnl'].mean():.2f}")
    print(f"  Avg duration: {df_live_positions['duration_minutes'].mean():.1f} minutes")
    print(f"  Win rate: {(df_live_positions['trade_pnl'] > 0).sum() / len(df_live_positions) * 100:.1f}%")
    
    print(f"\nDifference:")
    pnl_diff = df_live_positions['trade_pnl'].sum() - df_backtest_positions['trade_pnl'].sum()
    print(f"  P&L difference (Live - Backtest): ${pnl_diff:.2f}")
    
    if abs(df_backtest_positions['trade_pnl'].sum()) > 0:
        pct_diff = (pnl_diff / abs(df_backtest_positions['trade_pnl'].sum())) * 100
        print(f"  Percentage difference: {pct_diff:.2f}%")
    
    print(f"{'='*60}")
else:
    print("Insufficient position data to plot P&L comparison")

In [ ]:
# Function to load tick data for a time range
def load_trade_ticks(catalog, instrument_id, start_time, end_time, buffer_minutes=5):
    """
    Load trade tick data from catalog for a specific time window.
    
    Args:
        catalog: ParquetDataCatalog instance
        instrument_id: InstrumentId
        start_time: datetime (timezone-naive, local NY time)
        end_time: datetime (timezone-naive, local NY time)
        buffer_minutes: Minutes to add before/after for context
    
    Returns:
        DataFrame with tick data
    """
    # Add buffer
    buffer = pd.Timedelta(minutes=buffer_minutes)
    buffered_start = start_time - buffer
    buffered_end = end_time + buffer
    
    # IMPORTANT: Catalog expects UTC times, but our start/end are in NY local time
    # Convert NY local time to UTC for the catalog query
    buffered_start_utc = pd.Timestamp(buffered_start, tz='America/New_York').tz_convert('UTC').tz_localize(None)
    buffered_end_utc = pd.Timestamp(buffered_end, tz='America/New_York').tz_convert('UTC').tz_localize(None)
    
    try:
        # Load trade ticks from catalog (using UTC times)
        ticks = catalog.trade_ticks(
            instrument_ids=[instrument_id],
            start=buffered_start_utc,
            end=buffered_end_utc
        )
        
        if len(ticks) == 0:
            print(f"No ticks found for {buffered_start_utc} to {buffered_end_utc} (UTC)")
            return pd.DataFrame()
        
        # Convert to DataFrame
        tick_data = []
        for tick in ticks:
            # ts_init is in nanoseconds, convert to datetime (UTC -> NY local)
            tick_time = pd.to_datetime(tick.ts_init, unit='ns', utc=True).tz_convert('America/New_York').tz_localize(None)
            
            # Filter to our exact time range (with buffer, in NY local time)
            if buffered_start <= tick_time <= buffered_end:
                tick_data.append({
                    'time': tick_time,
                    'price': float(tick.price),
                    'size': float(tick.size) if hasattr(tick, 'size') else 0
                })
        
        df = pd.DataFrame(tick_data)
        return df.sort_values('time') if len(df) > 0 else df
    
    except Exception as e:
        print(f"Error loading ticks: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

print("Tick data loader function defined (fixed UTC/NY timezone conversion)")

### Define Tick Data Loader
Function to load trade ticks from the catalog for a specific time window.

**Key Operations**:
1. Add time buffer (default 1 minute) before entry and after exit for context
2. Convert NY local time → UTC for catalog query (catalog expects UTC)
3. Load ticks from catalog
4. Convert tick timestamps from UTC → NY local for display
5. Filter to exact time range

**Timezone Handling**: Critical to maintain consistency between catalog queries (UTC) and display (NY local time).

### Create Side-by-Side Position Comparison Plots

Generate interactive Plotly visualizations showing tick-level price movements for each position.

**Layout**:
- **Rows**: One row per position (up to 10 positions)
- **Columns**: 2 columns (Backtest left | Live right)
- **Height**: 450px per position for clear visualization
- **Spacing**: Minimal vertical spacing (3%) to maximize chart area

**For Each Position**:
1. Load tick data with 1-minute buffer before/after position
2. Plot tick prices as line+markers (blue for backtest, red for live)
3. Add vertical lines marking entry (green) and exit (red)
4. Display timestamps in HH:MM:SS format

**Interpretation**:
- Overlapping patterns → Same market conditions experienced
- Diverging patterns → Different data or timing issues
- Similar volatility → Consistent market behavior

In [ ]:
# ============================================================================
# CONFIGURE WHICH POSITIONS TO COMPARE
# ============================================================================
# Specify the 0-indexed position numbers you want to plot
backtest_position_idx = 0  # Which backtest position to plot (0 = first position)
live_position_idx = 0      # Which live position to plot (0 = first position)

# ============================================================================

# Get number of positions available
n_backtest = len(df_backtest_positions)
n_live = len(df_live_positions)

print(f"Available positions:")
print(f"  Backtest: {n_backtest} positions")
print(f"  Live: {n_live} positions")

if n_backtest == 0 or n_live == 0:
    print("\n❌ No positions found to plot")
elif backtest_position_idx >= n_backtest:
    print(f"\n❌ Backtest position index {backtest_position_idx} out of range (0-{n_backtest-1})")
elif live_position_idx >= n_live:
    print(f"\n❌ Live position index {live_position_idx} out of range (0-{n_live-1})")
else:
    # Get the selected position data
    bt_pos = df_backtest_positions.iloc[backtest_position_idx]
    live_pos = df_live_positions.iloc[live_position_idx]
    
    print(f"\n{'='*70}")
    print(f"COMPARING POSITIONS")
    print(f"{'='*70}")
    print(f"Backtest Position {backtest_position_idx + 1}/{n_backtest}:")
    print(f"  Time: {bt_pos['entry_time'].strftime('%H:%M:%S')} to {bt_pos['exit_time'].strftime('%H:%M:%S')}")
    print(f"  P&L: ${bt_pos['trade_pnl']:.2f}")
    print(f"\nLive Position {live_position_idx + 1}/{n_live}:")
    print(f"  Time: {live_pos['entry_time'].strftime('%H:%M:%S')} to {live_pos['exit_time'].strftime('%H:%M:%S')}")
    print(f"  P&L: ${live_pos['trade_pnl']:.2f}")
    print(f"{'='*70}\n")
    
    # Load tick data for this position
    bt_ticks = load_trade_ticks(catalog, instrument_id, bt_pos['entry_time'], bt_pos['exit_time'], buffer_minutes=1)
    live_ticks = load_trade_ticks(catalog, instrument_id, live_pos['entry_time'], live_pos['exit_time'], buffer_minutes=1)
    
    # Create side-by-side plotly subplots
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            f'Backtest Position {backtest_position_idx + 1}/{n_backtest}', 
            f'Live Position {live_position_idx + 1}/{n_live}'
        ),
        horizontal_spacing=0.08
    )
    
    # Plot backtest ticks (left subplot)
    if len(bt_ticks) > 0:
        fig.add_trace(
            go.Scatter(
                x=bt_ticks['time'],
                y=bt_ticks['price'],
                mode='lines+markers',
                name='Backtest Price',
                line=dict(color='cornflowerblue', width=2),
                marker=dict(size=4),
                hovertemplate='<b>Time:</b> %{x|%H:%M:%S}<br><b>Price:</b> $%{y:.2f}<extra></extra>',
                legendgroup='backtest',
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Add entry and exit markers using add_shape instead of add_vline
        fig.add_shape(
            type="line",
            x0=bt_pos['entry_time'], x1=bt_pos['entry_time'],
            y0=0, y1=1,
            yref="y domain",
            line=dict(color="green", width=2, dash="dash"),
            row=1, col=1
        )
        fig.add_annotation(
            x=bt_pos['entry_time'], y=1, yref="y domain",
            text="Entry", showarrow=False, yshift=10,
            row=1, col=1
        )
        
        fig.add_shape(
            type="line",
            x0=bt_pos['exit_time'], x1=bt_pos['exit_time'],
            y0=0, y1=1,
            yref="y domain",
            line=dict(color="red", width=2, dash="dash"),
            row=1, col=1
        )
        fig.add_annotation(
            x=bt_pos['exit_time'], y=1, yref="y domain",
            text="Exit", showarrow=False, yshift=10,
            row=1, col=1
        )
    else:
        print(f"  ⚠️ No backtest ticks found")
    
    # Plot live ticks (right subplot)
    if len(live_ticks) > 0:
        fig.add_trace(
            go.Scatter(
                x=live_ticks['time'],
                y=live_ticks['price'],
                mode='lines+markers',
                name='Live Price',
                line=dict(color='indianred', width=2),
                marker=dict(size=4),
                hovertemplate='<b>Time:</b> %{x|%H:%M:%S}<br><b>Price:</b> $%{y:.2f}<extra></extra>',
                legendgroup='live',
                showlegend=True
            ),
            row=1, col=2
        )
        
        # Add entry and exit markers using add_shape
        fig.add_shape(
            type="line",
            x0=live_pos['entry_time'], x1=live_pos['entry_time'],
            y0=0, y1=1,
            yref="y2 domain",
            line=dict(color="green", width=2, dash="dash"),
            row=1, col=2
        )
        fig.add_annotation(
            x=live_pos['entry_time'], y=1, yref="y2 domain",
            text="Entry", showarrow=False, yshift=10,
            row=1, col=2
        )
        
        fig.add_shape(
            type="line",
            x0=live_pos['exit_time'], x1=live_pos['exit_time'],
            y0=0, y1=1,
            yref="y2 domain",
            line=dict(color="red", width=2, dash="dash"),
            row=1, col=2
        )
        fig.add_annotation(
            x=live_pos['exit_time'], y=1, yref="y2 domain",
            text="Exit", showarrow=False, yshift=10,
            row=1, col=2
        )
    else:
        print(f"  ⚠️ No live ticks found")
    
    # Update layout
    fig.update_layout(
        title=f"{ticker} Position Tick Data Comparison<br><sub>Backtest #{backtest_position_idx + 1} vs Live #{live_position_idx + 1}</sub>",
        height=600,
        showlegend=True,
        hovermode='closest'
    )
    
    # Update x-axes
    fig.update_xaxes(title_text="Time", tickformat='%H:%M:%S', row=1, col=1)
    fig.update_xaxes(title_text="Time", tickformat='%H:%M:%S', row=1, col=2)
    
    # Update y-axes
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Price ($)", row=1, col=2)
    
    fig.show()

## Detailed Position Analysis with Trade Size Visualization

Focus on a specific position to examine the relationship between trade size and price movements. This helps identify whether outlier price spikes are caused by large block trades or small odd-lot trades executing at poor prices.

In [ ]:
"""
# Detailed tick analysis for a specific position with trade size visualization
import numpy as np

position_index = 7  # Position 8 (zero-indexed)

if position_index < len(df_live_positions):
    pos = df_live_positions.iloc[position_index]
    
    print(f"Analyzing Live Position {position_index + 1}")
    print(f"  Entry: {pos['entry_time']} at ${pos['entry_price']:.2f}")
    print(f"  Exit: {pos['exit_time']} at ${pos['exit_price']:.2f}")
    
    # Load tick data with 1-minute buffer
    ticks = load_trade_ticks(catalog, instrument_id, pos['entry_time'], pos['exit_time'], buffer_minutes=1)
    
    if len(ticks) > 0:
        print(f"  Loaded {len(ticks)} ticks")
        print(f"  Trade size range: {ticks['size'].min():.0f} - {ticks['size'].max():.0f} shares")
        print(f"  Price range: ${ticks['price'].min():.2f} - ${ticks['price'].max():.2f}")
        
        # Normalize trade sizes for marker sizing (scale to 2-20 pixel range)
        # Use log scale to avoid very large trades dominating
        min_size = 2
        max_size = 20
        size_normalized = ticks['size'].apply(lambda x: min_size + (max_size - min_size) * (np.log1p(x) / np.log1p(ticks['size'].max())))
        
        # Create figure
        fig = go.Figure()
        
        # Add tick data with size-based markers
        fig.add_trace(go.Scatter(
            x=ticks['time'],
            y=ticks['price'],
            mode='lines+markers',
            name='Trade Ticks',
            line=dict(color='indianred', width=1),
            marker=dict(
                size=size_normalized,
                color=ticks['size'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Trade<br>Size<br>(shares)"),
                opacity=0.7,
                line=dict(width=0.5, color='darkred')
            ),
            hovertemplate='<b>Time:</b> %{x}<br><b>Price:</b> $%{y:.2f}<br><b>Size:</b> %{marker.color:.0f} shares<extra></extra>'
        ))
        
        # Add entry and exit markers
        # Convert to string format for plotly compatibility
        entry_dt = pos['entry_time']
        exit_dt = pos['exit_time']
        
        # Use add_shape instead of add_vline to avoid timestamp arithmetic issues
        fig.add_shape(
            type="line",
            x0=entry_dt, x1=entry_dt,
            y0=0, y1=1,
            yref="paper",
            line=dict(color="green", width=2, dash="dash")
        )
        fig.add_annotation(
            x=entry_dt, y=1,
            yref="paper",
            text="ENTRY",
            showarrow=False,
            yshift=10
        )
        
        fig.add_shape(
            type="line",
            x0=exit_dt, x1=exit_dt,
            y0=0, y1=1,
            yref="paper",
            line=dict(color="red", width=2, dash="dash")
        )
        fig.add_annotation(
            x=exit_dt, y=1,
            yref="paper",
            text="EXIT",
            showarrow=False,
            yshift=10
        )
        
        # Update layout
        fig.update_layout(
            title=f"{ticker} - Live Position {position_index + 1} Tick Data (Trade Size Visualization)",
            xaxis_title='Time',
            yaxis_title='Price ($)',
            height=800,
            width=1500,
            hovermode='closest',
            xaxis=dict(tickformat='%H:%M:%S')
        )
        
        fig.show()
        
        # Show summary statistics
        print(f"\nTrade Size Statistics:")
        print(f"  Mean: {ticks['size'].mean():.0f} shares")
        print(f"  Median: {ticks['size'].median():.0f} shares")
        print(f"  Std Dev: {ticks['size'].std():.0f} shares")
        print(f"  Total Volume: {ticks['size'].sum():.0f} shares")
    else:
        print("  No tick data found for this position")
else:
    print(f"Position {position_index + 1} not found")
"""